In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install -U transformers datasets evaluate accelerate
!pip install scikit-learn
!pip install tensorboard

In [3]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    DataCollatorWithPadding,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    pipeline,
)

import evaluate
import glob
import numpy as np

In [4]:
import pandas as pd
# df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/hackathon/data/dark.csv')

In [5]:
# df.head()

,text,label
0,FLASH SALE | LIMITED TIME ONLY Shop Now,1
1,Pillowcases & Shams,0
2,Write a review,0
3,"To start your return, simply click on the foll...",0
4,newsletter signup (privacy policy),0


In [6]:
BATCH_SIZE = 32
NUM_PROCS = 32
LR = 0.00005
EPOCHS = 5
MODEL = 'bert-base-uncased'
OUT_DIR = '/content/drive/MyDrive/Colab Notebooks/hackathon/models'

In [24]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/hackathon/data/dark.csv')
df['label'] = df['label'].astype('int')

# Sample indices for train, validation, and test sets
train_indices = df.sample(frac=0.8, random_state=42).index
valid_indices = df.sample(frac=0.1, random_state=42).index
test_indices = df.index.difference(train_indices).difference(valid_indices)

# Filter rows based on indices
train_dataset = df[df.index.isin(train_indices)][['text', 'label']]
valid_dataset = df[df.index.isin(valid_indices)][['text', 'label']]
test_dataset = df[df.index.isin(test_indices)][['text', 'label']]


In [15]:
print(train_dataset.iloc[1])

text     Write a review
label                 0
Name: 2, dtype: object


In [26]:
from datasets import Dataset

# Preprocess the datasets
train_dataset = Dataset.from_pandas(train_dataset)
valid_dataset = Dataset.from_pandas(valid_dataset)
test_dataset = Dataset.from_pandas(test_dataset)

train_dataset = train_dataset.map(preprocess_function, batched=True)
valid_dataset = valid_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1885 [00:00<?, ? examples/s]

Map:   0%|          | 0/236 [00:00<?, ? examples/s]

Map:   0%|          | 0/471 [00:00<?, ? examples/s]

In [27]:
id2label = {
    0: "not_dark",
    1: "urgency",
    2: "scarcity",
    3: "mis_direction",
    4: "social_proof",
    5: "sneaking",
    6: "obstruction",
    7: "forced_action"
}

label2id = {label: idx for idx, label in id2label.items()}

In [17]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [18]:
def preprocess_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
    )

In [28]:
tokenized_train = train_dataset.map(
    preprocess_function,
    batched=True,
    batch_size=BATCH_SIZE,
    num_proc=NUM_PROCS
)

tokenized_valid = valid_dataset.map(
    preprocess_function,
    batched=True,
    batch_size=BATCH_SIZE,
    num_proc=NUM_PROCS
)

tokenized_test = test_dataset.map(
    preprocess_function,
    batched=True,
    batch_size=BATCH_SIZE,
    num_proc=NUM_PROCS
)

Map (num_proc=32):   0%|          | 0/1885 [00:00<?, ? examples/s]

Map (num_proc=32):   0%|          | 0/236 [00:00<?, ? examples/s]

Map (num_proc=32):   0%|          | 0/471 [00:00<?, ? examples/s]

In [29]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [30]:
tokenized_sample = preprocess_function(train_dataset[0])
print(tokenized_sample)

{'input_ids': [101, 5956, 5096, 1064, 3132, 2051, 2069, 4497, 2085, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


metrics

In [31]:
accuracy = evaluate.load('accuracy')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [32]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL,
    num_labels=8,
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=OUT_DIR,
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=3,
    report_to='tensorboard',
    fp16=False  # Set to True for mixed-precision training
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

history = trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
trainer.evaluate(tokenized_test)

In [ ]:
AutoModelForSequenceClassification.from_pretrained(f"arxiv_bert/checkpoint-4440")

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
classify = pipeline(task='text-classification', model=model, tokenizer=tokenizer)

all_files = glob.glob('inference_data/*')
for file_name in all_files:
    file = open(file_name)
    content = file.read()
    print(content)
    result = classify(content)
    print('PRED: ', result)
    print('GT: ', file_name.split('_')[-1].split('.txt')[0])
    print('\n')
